In [1]:
import os
import cv2
import numpy as np
from glob import glob

# 定义源文件夹和目标文件夹路径
source_folder = "D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/cam0/data"  # 左图文件夹
right_source_folder = "D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/cam1/data"  # 右图文件夹
target_folder = "D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/"  # 深度图保存文件夹

# 如果目标文件夹不存在，则创建它
os.makedirs(target_folder, exist_ok=True)

# 加载相机内参和外参（这里用的是之前的示例值）
K_left  = np.array([[667.3003, 0, 596.8533],
                    [0, 670.0276, 395.0267],
                    [0, 0, 1]])  # 左相机的内参矩阵
K_right = np.array([[673.4658, 0, 602.4980],
                    [0, 675.6429, 396.7246],
                    [0, 0, 1]])  # 右相机的内参矩阵
R = np.eye(3)  # 旋转矩阵
T = np.array([-120, 0, 0])  # 平移向量

# 使用 StereoBM 或 StereoSGBM 算法对象
stereo = cv2.StereoBM_create(numDisparities=64, blockSize=15)

# 获取所有左图的文件路径
left_image_paths = sorted(glob(os.path.join(source_folder, "*.jpg")))
right_image_paths = sorted(glob(os.path.join(right_source_folder, "*.jpg")))

# 过滤掉没有右图的文件
paired_image_paths = [(l, r) for l, r in zip(left_image_paths, right_image_paths) if os.path.isfile(r)]

# 计算深度图并保存
def compute_and_save_depth(left_image_path, right_image_path, target_image_path):
    # 读取左右图像
    left_image = cv2.imread(left_image_path, cv2.IMREAD_GRAYSCALE)
    right_image = cv2.imread(right_image_path, cv2.IMREAD_GRAYSCALE)

    # 计算视差图
    disparity = stereo.compute(left_image, right_image).astype(np.float32) / 16.0

    # 将视差转换为深度图
    focal_length = 210
    baseline = 120
    depth_map = np.zeros_like(disparity)
    
    # 深度值计算（避免分母为零）
    valid_pixels = disparity > 0
    depth_map[valid_pixels] = (focal_length * baseline) / disparity[valid_pixels]

    # 处理深度图中的无效值
    depth_map[depth_map > 10000] = 0  # 过滤远处的无效值

    # 归一化深度图到 [0, 65535] 范围
    depth_map_normalized = cv2.normalize(depth_map, None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX)

    # 保存深度图
    cv2.imwrite(target_image_path, depth_map_normalized.astype(np.uint16))
    print(f"Depth map for {os.path.basename(left_image_path)} saved to {target_image_path}")

# 主流程
for left_image_path, right_image_path in paired_image_paths:
    # 构建保存路径
    image_name = os.path.basename(left_image_path)
    target_image_path = os.path.join(target_folder, os.path.splitext(image_name)[0] + '.pgm')

    # 计算并保存深度图
    compute_and_save_depth(left_image_path, right_image_path, target_image_path)

Depth map for 17333604000000000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000000000000.pgm
Depth map for 17333604000050000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000050000000.pgm
Depth map for 17333604000100000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000100000000.pgm
Depth map for 17333604000150000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000150000000.pgm
Depth map for 17333604000200000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000200000000.pgm
Depth map for 17333604000250000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000250000000.pgm
Depth map for 17333604000300000000.jpg saved to D:/docker_test/catkin_ws/src/AirSlam/dataroot/room2/depth/17333604000300000000.pgm
Depth map for 17333604000350000000.jpg saved to D:/docker_test/catkin_ws/src/AirSla

In [2]:
# import cv2
# import numpy as np
# import time
# import os
# import math
# from glob import glob
# # 左相机内参
# left_camera_matrix = np.array([[667.3003, 0.0, 596.8533],
#                                 [0., 670.0276, 395.0267],
#                                 [0., 0., 1.]])
 
# # 左相机畸变系数:[k1, k2, p1, p2, k3]
# left_distortion = np.array([[0.0895, -0.1429, 0.0029, -0.0001192, 0.0219]])
 
# # 右相机内参
# right_camera_matrix = np.array([[673.4658, 0.0, 602.4980],
#                                 [0., 675.6429, 396.7246],
#                                 [0., 0., 1.]])
# # 右相机畸变系数:[k1, k2, p1, p2, k3]                                          
# right_distortion = np.array([[0.1034, -0.189, 0.0024, -0.00063025, 0.0593]])

# # 旋转矩阵
# R = np.array([[0.999962872853149, 0.00187779299260463, -0.00840992323112715],
#             [ -0.0018408858041373, 0.999988651353238, 0.00439412154902114],
#             [ 0.00841807904053251, -0.00437847669953504, 0.999954981430194]])
 
# # 平移向量
# T = np.array([[-120.326603502087], [0], [0]])

# # 图像尺寸
# size = (1280, 720)

# # 立体校正
# R1, R2, P1, P2, Q, validPixROI1, validPixROI2 = cv2.stereoRectify(left_camera_matrix, left_distortion,
#                                                                 right_camera_matrix, right_distortion, size, R, T)

# # 校正查找映射表
# left_map1, left_map2 = cv2.initUndistortRectifyMap(left_camera_matrix, left_distortion, R1, P1, size, cv2.CV_16SC2)
# right_map1, right_map2 = cv2.initUndistortRectifyMap(right_camera_matrix, right_distortion, R2, P2, size, cv2.CV_16SC2)

# # 定义源文件夹和目标文件夹路径
# source_folder = "D:/docker_test/catkin_ws/src/AirSlam/dataroot/room/cam0/data"  # 左图文件夹
# right_source_folder = "D:/docker_test/catkin_ws/src/AirSlam/dataroot/room/cam1/data"  # 右图文件夹
# target_folder = "D:/docker_test/catkin_ws/src/AirSlam/dataroot/room/depth/"  # 深度图保存文件夹
# # 获取所有左图的文件路径
# left_image_paths = sorted(glob(os.path.join(source_folder, "*.jpg")))
# right_image_paths = sorted(glob(os.path.join(right_source_folder, "*.jpg")))

# # 过滤掉没有右图的文件
# paired_image_paths = [(l, r) for l, r in zip(left_image_paths, right_image_paths) if os.path.isfile(r)]
# # 如果目标文件夹不存在，则创建它
# os.makedirs(target_folder, exist_ok=True)
# imageWidth = 1280
# imageHeight = 720

# def compute_and_save_depth(left_image_path, right_image_path, target_image_path):
#     imgL = cv2.imread(left_image_path)
#     imgR = cv2.imread(right_image_path)

#     img1_rectified = cv2.remap(imgL, left_map1, left_map2, cv2.INTER_LINEAR)
#     img2_rectified = cv2.remap(imgR, right_map1, right_map2, cv2.INTER_LINEAR)

#     blockSize = 8
#     img_channels = 3
#     stereo = cv2.StereoSGBM_create(minDisparity=1,
#                                 numDisparities=64,
#                                 blockSize=blockSize,
#                                 P1=8 * img_channels * blockSize * blockSize,
#                                 P2=32 * img_channels * blockSize * blockSize,
#                                 disp12MaxDiff=-1,  
#                                 preFilterCap=140,
#                                 uniquenessRatio=1,      
#                                 speckleWindowSize=100,
#                                 speckleRange=100,
#                                 mode=cv2.STEREO_SGBM_MODE_HH)
#     # 计算视差
#     disparity = stereo.compute(img1_rectified, img2_rectified)

#     # 归一化函数算法，生成深度图（灰度图）
#     disp = cv2.normalize(disparity, disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
#     cv2.imwrite(target_image_path, disp)

# for left_image_path, right_image_path in paired_image_paths:
#     # 构建保存路径
#     image_name = os.path.basename(left_image_path)
#     target_image_path = os.path.join(target_folder, os.path.splitext(image_name)[0] + '.pgm')

#     # 计算并保存深度图
#     compute_and_save_depth(left_image_path, right_image_path, target_image_path)

In [3]:
# import cv2
# import numpy as np

# # 读取左图和右图
# left_image = cv2.imread("/home/jg/图片/双目/cali_left/1.jpg", cv2.IMREAD_GRAYSCALE)
# right_image = cv2.imread("/home/jg/图片/双目/cali_right/1.jpg", cv2.IMREAD_GRAYSCALE)

# # 加载相机内参和外参（示例）
# K_left = np.array([[161.95, 0, 301.1995],
#                    [0, 161.15, 218.5885],
#                    [0, 0, 1]])  # 左相机的内参矩阵
# K_right = np.array([[161.59, 0, 296.868],
#                     [0, 161.16, 217.8686],
#                     [0, 0, 1]])  # 右相机的内参矩阵
# # 相机间的旋转和平移矩阵（从左相机到右相机的变换）
# R = np.eye(3)  # 旋转矩阵
# T = np.array([-0.12, 0, 0])  # 平移向量

# # 使用 StereoSGBM 算法进行立体匹配
# stereo = cv2.StereoSGBM_create(minDisparity=0, numDisparities=64, blockSize=5)
# disparity = stereo.compute(left_image, right_image) / 16.0

# # 将视差转换为深度图
# focal_length = 2666.667  # 假设焦距相同
# baseline = 0.12  # 假设基线为 0.1米

# # 计算深度图：深度 D = (focal_length * baseline) / disparity
# depth_map = (focal_length * baseline) / disparity.astype(np.float32)

# # 处理深度图中的无效值
# depth_map[disparity == 0] = 0

# # 将深度图保存为 PGM 文件
# depth_map_uint16 = (depth_map * 1000).astype(np.uint16)  # 将深度图转换为毫米单位
# cv2.imwrite("depth_map.pgm", depth_map_uint16)
